In [1]:
# 課題6との違い


# コマンド入力を元に宝石を移動させる

In [8]:
import random

In [2]:
# 関数宣言
import random 


ELEMENT_SYMBOLS ={
    '火': '$',
    '水': '~',
    '風': '@',
    '土': '#',
    '命': '&',
    '無': ' '
}
ELEMENT_COLORS={
    '火': '41',
    '水': '44',
    '風': '42',
    '土': '43',
    '命': '45',
    '無': '47'
}
GEM_ELEMENT = ['火', '水', '風', '土', '命'] 
COMMAND = "ABCDEFGHIJKLMN"

def print_monster_name(monster):
    global ELEMENT_SYMBOLS, ELEMENT_COLORS
    element = monster['element']
    symbol = ELEMENT_SYMBOLS[element]
    color = ELEMENT_COLORS[element]

    print(f'\033[{color}m{symbol}{monster["name"]}{symbol}\033[0m', end='')

    
    
def main():
    player_name = input('プレイヤー名を入力してください>>')
    if player_name == '':
        print('エラー:プレイヤー名を入力してください。')
    else:
        print('### Puzzle & Monsters ###')
        # 敵モンスター
        slime = {'name':'スライム', 'hp':100, 'max_hp':100, 'element':'水', 'ap':10, 'dp':1}
        goblin = {'name':'ゴブリン', 'hp':200, 'max_hp':200, 'element':'土', 'ap':20, 'dp':5}
        big_bat = {'name':'オオコウモリ', 'hp':300, 'max_hp':300, 'element':'風', 'ap':30, 'dp':10}
        wolf = {'name':'ウェアウルフ', 'hp':400, 'max_hp':400, 'element':'風', 'ap':40, 'dp':15}
        dragon = {'name':'ドラゴン', 'hp':600, 'max_hp':600, 'element':'火', 'ap':50, 'dp':20}

        #味方モンスター
        suzaku ={'name':'朱雀', 'hp':150, 'max_hp':150, 'element':'火', 'ap':25, 'dp':10}
        seiryu ={'name':'青龍', 'hp':150, 'max_hp':150, 'element':'風', 'ap':15, 'dp':10}
        byakko ={'name':'白虎', 'hp':150, 'max_hp':150, 'element':'土', 'ap':20, 'dp':5}
        genbu ={'name':'玄武', 'hp':150, 'max_hp':150, 'element':'水', 'ap':20, 'dp':15}

        # global slime,goblin, big_bat,wolf, dragon
        enemies =[slime, goblin, big_bat, wolf, dragon]
        # global suzaku, seiryu, byakko, genbu
        friends =[suzaku, seiryu, byakko, genbu]
        party = organize_party(player_name,friends)
        number = go_dungeon(party, enemies) 
        if number == 5:
            print('### GAME CLEARED!! ###')
        else:
            print('### GAME OVER!! ###')

        print(f'倒したモンスター数={number}')
        
def organize_party(player_name, friends):
    total_hp = sum([f['hp'] for f in friends])
    dp = [f['dp'] for f in friends]
    dp_avg = sum(dp)/len(friends)
    
    party={'player_name':player_name, 'friends':friends,
          'max_hp':total_hp, 'hp':total_hp, 'dp': dp_avg}
    return party

def show_party(party):
    print('<パーティー編成>----------------')
    
    for friend in party['friends']:
        print_monster_name(friend)
        print(f'HP={friend["hp"]} 攻撃={friend["ap"]} 防御={friend["dp"]}')
    print('-----------------------------')

def go_dungeon(party, enemies):
    
    print(f'{party["player_name"]}のパーティ(HP={party["max_hp"]})はダンジョンに到着した')
    show_party(party)
    # monsters= ['スライム', 'ゴブリン', 'オオコウモリ', 'ウェアウルフ', 'ドラゴン']
    win_flg = 0
    for enemy in enemies:
        # enemy_name = enemy['name']
        win_flg += do_battle(party, enemy)
        if party['hp'] <= 0:
            print(f'{party["player_name"]}はダンジョンから逃げ出しました')
            return win_flg
        else:
            print(f'{party["player_name"]}は更に奥に進んだ')
            print(f'=======================================')
            
    print(f'{party["player_name"]}はダンジョンを制覇した')
    return win_flg


def do_battle(party, enemy):
    enemy_name = enemy['name']
    print_monster_name(enemy)
    print(f'が現れた!')    
    gems = [] # random.choices(list(ELEMENT_SYMBOLS.keys()), k=14)
    gems = fill_gems(gems)
    
    battle_field ={
        'enemy': enemy,
        'party': party,
        'gems':gems
    }

    while True:
        on_player_turn(battle_field)
        if enemy['hp'] <= 0:
            break
        print("")
        on_enemy_turn(party, enemy)
        if party['hp'] <= 0:
            print("パーティのHPは０になりました")
            return 0
    
    print_monster_name(enemy)
    print(f'を倒した!')
    
    return 1



def show_battle_field(battle_field):
    print("バトルフィールド")

    enemy = battle_field['enemy']
    party = battle_field['party']
    print_monster_name(enemy)
    print(f"HP = {enemy['hp']} / {enemy['max_hp']}")
    print("    ")
    for f in party['friends']:
        print_monster_name(f)
        print("", end=" ")
    print("")
    
    print(f"HP = {party['hp']} / {party['max_hp']}")
    
    print("-"*20)
    for c in COMMAND:
        print(c, end=" ")
    print("")
    print_gems(battle_field['gems'])
    print("")
    print("-"*20)
    
    
    

def on_player_turn(battle_field):
    
    party, enemy = battle_field['party'], battle_field['enemy']
    print(f"【{party['player_name']}のターン(HP={party['hp']})】")
    
    show_battle_field(battle_field)
    
    command = input("コマンド?>>")
    while not check_valid_command(command):
        print('入力されたコマンドは不適切です')
        command = input("コマンド?>>")
        
    move_gem(command=command, gems=battle_field['gems'])
    
    evaluate_gems(battle_field, command)
    # do_attack(enemy, command)

    
    

def do_attack(enemy, command):
    damage = hash(command) % 50
    damage = int(random.uniform(damage-0.1*damage, damage+0.1*damage))
    print(f"ダミー攻撃で相手に{damage}のダメージを与えた")
    enemy['hp'] -= damage
    if enemy['hp'] <= 0:
        enemy['hp'] = 0

def on_enemy_turn(party, enemy):
    print(f"【{enemy['name']}のターン(HP={enemy['hp']})】")
        
    do_enemy_attack(party)
    
        
def do_enemy_attack(party):
    damage = 200
    party['hp'] -= damage
    print(f"ダミー攻撃で相手に{damage}のダメージを与えた")
    print(f"")
    if party['hp'] <0:
        party['hp'] = 0
    

def fill_gems(gems):
    global ELEMENT_SYMBOLS
    new_gems = gems[:]
    now_gems_len = len(gems)
    required_gems = 14 - now_gems_len
    new_gems += [  random.choice(GEM_ELEMENT) for i in  range(required_gems)]
    
    
    return new_gems



def print_gems(gems):
    gems_with_color_code = [f'\033[{ELEMENT_COLORS[f]}m{ELEMENT_SYMBOLS[f]}\033[0m' for f in gems]
    for i in gems_with_color_code:
        print(i, end=' ')

def evaluate_gems(battle_field, command):
    enemy, party = battle_field['enemy'], battle_field['party']
    do_attack(enemy, command)

def swap_gems(gems, designated_gem_idx, direction):
    """
    designated_gem_idx：　指定された宝石の添字
    direction:  右なら１、　左なら−１に動かす
    """
    tmp = gems[designated_gem_idx+ direction]
    # print(tmp, designated_gem_idx)
    gems[designated_gem_idx+direction] = gems[designated_gem_idx]
    gems[designated_gem_idx] = tmp
    
def check_valid_command(command):
    """
        正しいコマンドなら　True
        不適切なコマンドなら　False
    """
    if len(command) == 2:
        if command[0] in COMMAND and command[1] in COMMAND and command[0] != command[1]:
            return True
        else:
            return False
    else:
        return False
    
    

def move_gem(command, gems):
    global COMMAND
    start_index = COMMAND.index(command[0])
    end_index = COMMAND.index(command[1])

    # start_index < end_indexと仮定
    if start_index < end_index:
        direc = 1
        num = end_index - start_index
    else:
        direc = -1
        num = start_index - end_index

    print_gems(gems)
    print("")
    print("")


    for i in range(0, num):
        swap_gems(gems,start_index+(i*direc), direction= direc)
        print_gems(gems)
        print("")
        print("")
    # print(COMMAND[start_index+i], COMMAND[start_index+i+1])
# main()

In [2]:
tmp = list("ABEDEF")
print(tmp)

b_index = tmp.index("B")
e_index = tmp.index("E")

print(b_index, e_index)

['A', 'B', 'E', 'D', 'E', 'F']
1 2


In [ ]:
main()

プレイヤー名を入力してください>> すっとん


### Puzzle & Monsters ###
すっとんのパーティ(HP=600)はダンジョンに到着した
<パーティー編成>----------------
$朱雀$HP=150 攻撃=25 防御=10
@青龍@HP=150 攻撃=15 防御=10
#白虎#HP=150 攻撃=20 防御=5
~玄武~HP=150 攻撃=20 防御=15
-----------------------------
~スライム~が現れた!
【すっとんのターン(HP=600)】
バトルフィールド
~スライム~HP = 100 / 100
    
$朱雀$ @青龍@ #白虎# ~玄武~ 
HP = 600 / 600
--------------------
A B C D E F G H I J K L M N 
@ # # $ # ~ $ ~ # ~ $ ~ & # 
--------------------


コマンド?>> AC


@ # # $ # ~ $ ~ # ~ $ ~ & # 

# @ # $ # ~ $ ~ # ~ $ ~ & # 

# # @ $ # ~ $ ~ # ~ $ ~ & # 

ダミー攻撃で相手に10のダメージを与えた

【スライムのターン(HP=90)】
ダミー攻撃で相手に200のダメージを与えた

【すっとんのターン(HP=400)】
バトルフィールド
~スライム~HP = 90 / 100
    
$朱雀$ @青龍@ #白虎# ~玄武~ 
HP = 400 / 600
--------------------
A B C D E F G H I J K L M N 
# # @ $ # ~ $ ~ # ~ $ ~ & # 
--------------------


コマンド?>> CD


# # @ $ # ~ $ ~ # ~ $ ~ & # 

# # $ @ # ~ $ ~ # ~ $ ~ & # 

ダミー攻撃で相手に1のダメージを与えた

【スライムのターン(HP=89)】
ダミー攻撃で相手に200のダメージを与えた

【すっとんのターン(HP=200)】
バトルフィールド
~スライム~HP = 89 / 100
    
$朱雀$ @青龍@ #白虎# ~玄武~ 
HP = 200 / 600
--------------------
A B C D E F G H I J K L M N 
# # $ @ # ~ $ ~ # ~ $ ~ & # 
--------------------


'ABCDEFGHIJKLMN'

In [65]:
for i in range(end_index - start_index):
    print(1)

1
1


In [55]:
gems

a = fill_gems(gems)
len(a)


print_gems(a)

  @ @ ~ ~ & & $ @ $   @ @ & 